# Feature selection

USPTO provides several research datasets containing information about the patents and the application process. The enormous amount of data available required an advanced and complex data cleaning phase to extract meaningful features. Google Cloud Services, in particular the service BigQuery was extensively used to analyze terabytes of data to extract the features. 

### Overview of the datasets
We used the following USPTO datasets: 
- [Patent Examination Research Dataset (PaTex)](https://www.uspto.gov/ip-policy/economic-research/research-datasets/patent-examination-research-dataset-public-pair)
- [Patent Claims Research Dataset](https://www.uspto.gov/ip-policy/economic-research/research-datasets/patent-claims-research-dataset)

We aim to build a dataset containing the following features

| Feature | Description |
|---------|-------------|
|`application_number`| (will not be used for model training) |
|`approved`| `1` if approved, `0` if rejected |
|`examiner_art_unit` | Art unit of the examiner |
|`uspc_class`| Class of USPC | 
|`uspc_subclass` | Subclass of USPC |
|`inventors_count` | Number of inventors |
|`inventors_avg_approval_rate` | Average approval rate of inventors | 
|`inventors_avg_patents` | Average number of patents per inventors
|`inventors_country` | The country from which the majority of inventors come from
|`attorney_count` | Number of attorneys |
|`attorney_avg_approval_rate` | Average approval rate of attorneys | 
|`attorney_avg_patents` | Average number of patents per attorney |


The dataset will contain: 
- Patents that were approved between 2010 and 2015 that were not rejected at the first attempt for non-novelty, for which we have information about the attorneys, the inventors and the patent claims
- Applications that were rejected at the first attempt between 2010 and 2015, for which we have information about the attorneys, the inventors and the patent claims

We will consider only applications in which the application type is `REGULAR` and the invetion type is `UTILITY`.

##### Note
The following SQL queries were executed on BigQuery on USPTO research datasets. Those datasets are already loaded on Google Cloud and have to be activated [here](https://console.cloud.google.com/marketplace/browse?q=google+patents+public+datasets&filter=solution-type:dataset)  
The result of `SELECT` queries were saved in the tables listed in comments over the queries.  
We runned several tests to verify that the results are accurate and corrent. The tests are not reported in this notebook.

### Office actions filtering
Let's start by filtering only approved applications between 2010 and 2015. 

In [ ]:
# application_data_approved_patents
SELECT application_number, patent_number, filing_date, patent_issue_date, examiner_art_unit, uspc_class, uspc_subclass 
    FROM `patents-public-data.uspto_oce_pair.application_data` 
    WHERE patent_issue_date >= "2010-01-01" AND patent_issue_date <= "2015-12-31" AND application_type = "REGULAR" AND invention_subject_matter = "UTL"

With regards to rejected applications, we firstly need to consider the first office action for each application and select non-final rejections.     

In [ ]:
# first_office_actions
SELECT * FROM `patents-public-data.uspto_oce_office_actions.office_actions` A, 
    (SELECT B.app_id, MIN(mail_dt) as min_date FROM `patents-public-data.uspto_oce_office_actions.office_actions` B
    GROUP BY B.app_id) B
    WHERE mail_dt >= "2010-01-01" AND mail_dt <= "2015-12-31" AND
    A.app_id = B.app_id AND A.mail_dt = B.min_date 


# first_office_actions_2010_2015_non-novelty
SELECT * FROM `ml-project2-333508.office-actions.first_office_actions` A
    WHERE rejection_102 = 1  

# application_data_rejected_non-novelty
SELECT A.* FROM  `patents-public-data.uspto_oce_pair.application_data` A, 
    `ml-project2-333508.office-actions.first_office_actions_2010_2015_non-novelty` B
    WHERE A.application_type = "REGULAR" AND invention_subject_matter = "UTL" AND
    A.application_number = B.app_id    

### Patent claims
Let's now extract the patent claims for the selected applications. We will later remove the applications for which we can't find any patent claim. 

In [ ]:
# approved_patent_claims
SELECT C.pat_no, C.claim_no, C.claim_txt 
    FROM `patents-public-data.uspto_oce_claims.patent_claims_fulltext` C 
    WHERE C.pat_no IN (SELECT patent_number FROM `ml-project2-333508.office_actions.application_data_approved_patents`) ORDER BY pat_no, claim_no

# rejected_patent_claims
SELECT C.pat_no, C.claim_no, C.claim_txt 
    FROM `patents-public-data.uspto_oce_claims.patent_claims_fulltext` C 
    WHERE C.pat_no IN (SELECT patent_number FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty`) ORDER BY pat_no, claim_no

Let's now remove from `application_data` the applications that don't have any patent claim available.   

In [ ]:
# application_data_approved_patents_with_claim
SELECT A.* 
    FROM `ml-project2-333508.office_actions.application_data_approved_patents` A 
    WHERE A.patent_number IN (SELECT pat_no FROM `ml-project2-333508.patent_claims_fulltext.approved_patents_claims`) ORDER BY A.patent_number

# application_data_rejected_non-novelty_with_claim
SELECT A.* 
    FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty` A 
    WHERE A.patent_number IN (SELECT appl_id FROM `ml-project2-333508.patent_claims_fulltext.rejected_patents_claims`) ORDER BY A.patent_number


There may exist some applications that were firstly rejected for non-novelty (with a non-final rejected) and that were later approved. We will remove those applications from the `approved` dataset and keep them in the `rejected` one. 

In [ ]:
# application_data_approved_filtered
SELECT A.* 
    FROM `ml-project2-333508.office_actions.application_data_approved_patents_with_claim` A 
    WHERE A.application_number NOT IN 
        (SELECT B.application_number FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty_with_claim` B ) ORDER BY application_number

We also have to compute again the table containing the patent claims for approved applications. 

In [ ]:
SELECT A.*
    FROM `ml-project2-333508.patent_claims_fulltext.rejected_patents_claims` A, 
    WHERE A.appl_id IN (SELECT B.application_number FROM `ml-project2-333508.office_actions.application_data_approved_filtered`)

Let's now tidy up the tables containg patent claims. Each claims starts with the number of the claim like the following example: 

| pat_no | claim_no | clam_txt | 
|----------|------|-----|
|7640599| 1 | 1. A bib for illustrating an infant's last feeding time, said bib comprising \[...\] |
|7640599| 2 | 2. A bib for illustrating an infant's last feeding time, according to claim 1 \[...\] |
| 7640599 | \[...\] | \[...\] |

In [ ]:
# approved_patents_claims_filtered_cleaned
SELECT pat_no, claim_no, SUBSTR(claim_txt, INSTR(claim_txt, ".") + 2) AS claim_txt 
    FROM `ml-project2-333508.patent_claims_fulltext.approved_patents_claims_filtered` ORDER BY pat_no, CAST(claim_no AS INT)

# rejected_patents_non-novelty_claims_cleaned
SELECT appl_id, claim_no, SUBSTR(claim_txt, INSTR(claim_txt, ".") + 2) AS claim_txt 
    FROM `ml-project2-333508.patent_claims_fulltext.rejected_patents_non-novelty_claims` ORDER BY appl_id, CAST(claim_no AS INT)

### Inventors
As stated in our paper, we believe that the history of an inventor will influence the outcome of future applications. 
For each patent application, we will compute: 
- The number of inventors
- The average approval rate for the inventors
- The average number of patent applications submitted by the inventors

Those informations will be computed using applications between 2010 and 2015. 

Let's start by counting the number of approved and rejected patents for each inventor. 

In [ ]:
# inventors_rejected_count
SELECT inventor_name_first, inventor_name_middle, inventor_name_last, inventor_country_code, COUNT(DISTINCT application_number)  as rejected_patents FROM `ml-project2-333508.inventors.inventors_rejected_patents` 
    GROUP BY inventor_name_first, inventor_name_middle, inventor_name_last, inventor_country_code

# inventors_approved_count_filtered
SELECT inventor_name_first, inventor_name_middle, inventor_name_last, inventor_country_code, COUNT(DISTINCT application_number)  as approved_patents FROM `ml-project2-333508.inventors.inventors_approved_patents_filtered` 
    GROUP BY inventor_name_first, inventor_name_middle, inventor_name_last, inventor_country_code

# inventors_data_filtered_mixed (contains the information about the inventors that had both approved and rejected patents)
SELECT A.inventor_name_first, A.inventor_name_middle, A.inventor_name_last, A.inventor_country_code, (A.approved_patents + R.rejected_patents) as total_patents, ROUND((A.approved_patents/(A.approved_patents + R.rejected_patents))*100, 2) as approval_rate
    FROM `ml-project2-333508.inventors.inventors_approved_filtered_count` A, 
    `ml-project2-333508.inventors.inventors_rejected_count` R
    WHERE A.inventor_name_first = R.inventor_name_first AND ((A.inventor_name_middle = R.inventor_name_middle) OR (A.inventor_name_middle IS NULL AND R.inventor_name_middle IS NULL)) AND A.inventor_name_last = R.inventor_name_last AND
    ((A.inventor_country_code = R.inventor_country_code) OR (A.inventor_country_code IS NULL AND R.inventor_country_code IS NULL))

# inventors_data_filtered_approved (only those who only have approved patents)
SELECT A.inventor_name_first, A.inventor_name_middle, A.inventor_name_last, A.inventor_country_code, (A.approved_patents) as total_patents, 100.0 as approval_rate
    FROM `ml-project2-333508.inventors.inventors_approved_patents_filtered_count` A
    WHERE
    NOT EXISTS
    (SELECT R.inventor_name_first FROM `ml-project2-333508.inventors.inventors_rejected_count` R
    WHERE R.inventor_name_first = A.inventor_name_first AND ((R.inventor_name_middle = A.inventor_name_middle) OR (R.inventor_name_middle IS NULL AND A.inventor_name_middle IS NULL)) AND 
    R.inventor_name_last = A.inventor_name_last AND ((R.inventor_country_code = A.inventor_country_code) OR (R.inventor_country_code IS NULL AND A.inventor_country_code IS NULL))
    )

# inventors_data_filtered_rejected (only those who only have rejected patents)
SELECT A.inventor_name_first, A.inventor_name_middle, A.inventor_name_last, A.inventor_country_code, (A.rejected_patents) as total_patents, 0.0 as approval_rate
    FROM `ml-project2-333508.inventors.inventors_rejected_count` A
    WHERE
    NOT EXISTS
    (SELECT R.inventor_name_first FROM `ml-project2-333508.inventors.inventors_approved_patents_filtered_count` R
    WHERE R.inventor_name_first = A.inventor_name_first AND ((R.inventor_name_middle = A.inventor_name_middle) OR (R.inventor_name_middle IS NULL AND A.inventor_name_middle IS NULL)) AND 
    R.inventor_name_last = A.inventor_name_last AND ((R.inventor_country_code = A.inventor_country_code) OR (R.inventor_country_code IS NULL AND A.inventor_country_code IS NULL))
    )

# inventors_data_filtered (we merge all the previous tables)
(SELECT * FROM `ml-project2-333508.inventors.inventors_data_filtered_approved`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.inventors.inventors_data_filtered_mixed`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.inventors.inventors_data_filtered_rejected`)


Now we have a table (`inventors_data_filtered`) that contains the following columns: 
- `inventor_name_first`
- `inventors_name_middle`
- `inventor_name_last`
- `inventor_country_code`
- `total_patents` (patents approved + patents rejected for non-novelty at first attempt)
- `approval_rate` (percentage of approved patents)

We now build the features for each application

In [ ]:
# features_inventors_approved
SELECT A.application_number, AVG(I.approval_rate) as inventor_avg_approval_rate, AVG(I.total_patents) as inventor_avg_total_patents, COUNT(*) as inventors_number
    FROM `ml-project2-333508.office_actions.application_data_approved_patents_filtered` A, 
    `ml-project2-333508.inventors.inventors_data_filtered` I,
    `ml-project2-333508.inventors.inventors_approved_patents_filtered` I_A
    WHERE A.application_number = I_A.application_number AND
    I_A.inventor_name_first = I.inventor_name_first AND ((I_A.inventor_name_middle = I.inventor_name_middle) OR (I_A.inventor_name_middle IS NULL AND I.inventor_name_middle IS NULL)) AND I_A.inventor_name_last = I.inventor_name_last AND
    ((I_A.inventor_country_code = I.inventor_country_code) OR (I_A.inventor_country_code IS NULL AND I.inventor_country_code IS NULL) )
    GROUP BY A.application_number
    ORDER BY A.application_number

# features_inventors_rejected
SELECT A.application_number, AVG(I.approval_rate) as inventor_avg_approval_rate, AVG(I.total_patents) as inventor_avg_total_patents, COUNT(*) as inventors_number
    FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty_with_claim` A, 
    `ml-project2-333508.inventors.inventors_data_filtered` I,
    `ml-project2-333508.inventors.inventors_rejected_patents` I_A
    WHERE A.application_number = I_A.application_number AND
    I_A.inventor_name_first = I.inventor_name_first AND ((I_A.inventor_name_middle = I.inventor_name_middle) OR (I_A.inventor_name_middle IS NULL AND I.inventor_name_middle IS NULL)) AND I_A.inventor_name_last = I.inventor_name_last AND
    ((I_A.inventor_country_code = I.inventor_country_code) OR (I_A.inventor_country_code IS NULL AND I.inventor_country_code IS NULL) )
    GROUP BY A.application_number
    ORDER BY A.application_number

### Attorneys
We will do the same computations that we did with inventors. 
For each patent application, we will compute: 
- The number of attorneys
- The average approval rate for the attorneys
- The average number of patent applications submitted by the attorneys

Those informations will be computed using applications between 2010 and 2015. 

Let's start by counting the number of approved and rejected patents for each attorney. 

In [ ]:
# attorneys_approved_patents
SELECT application_number, atty_registration_number FROM `ml-project2-333508.attorneys.attorney_agent`A WHERE 
    EXISTS (SELECT B.application_number FROM `ml-project2-333508.office_actions.application_data_approved_patents_filtered` B WHERE B.application_number = A.application_number)

# attorneys_rejected_patents
SELECT application_number, atty_registration_number FROM `ml-project2-333508.attorneys.attorney_agent`A WHERE 
    EXISTS (SELECT B.application_number FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty_with_claim` B WHERE B.application_number = A.application_number)

# attorneys_approved_count
SELECT A.atty_registration_number, COUNT(DISTINCT A.application_number) as approved_patents FROM `ml-project2-333508.attorneys.attorneys_approved_patents` A 
    GROUP BY A.atty_registration_number 
    ORDER BY A.atty_registration_number

# attorneys_rejected_patents_count
SELECT A.atty_registration_number, COUNT(DISTINCT A.application_number) as rejected_patents FROM `ml-project2-333508.attorneys.attorneys_rejected_patents` A 
    GROUP BY A.atty_registration_number 
    ORDER BY A.atty_registration_number

# attorneys_data_mixed
SELECT A.atty_registration_number, (A.approved_patents + R.rejected_patents) as total_patents, ROUND((A.approved_patents/(A.approved_patents + R.rejected_patents))*100, 2) as approval_rate
    FROM `ml-project2-333508.attorneys.attorneys_approved_patents_count` A, 
    `ml-project2-333508.attorneys.attorneys_rejected_patents_count` R
    WHERE A.atty_registration_number = R.atty_registration_number

# attorneys_data_approved
SELECT A.atty_registration_number, (A.approved_patents) as total_patents, 100.0 as approval_rate
    FROM `ml-project2-333508.attorneys.attorneys_approved_patents_count` A
    WHERE
    NOT EXISTS
    (SELECT R.atty_registration_number FROM `ml-project2-333508.attorneys.attorneys_rejected_patents_count` R
    WHERE R.atty_registration_number = A.atty_registration_number)

# attorneys_data_rejected
SELECT A.atty_registration_number, (A.rejected_patents) as total_patents, 0.0 as approval_rate
    FROM `ml-project2-333508.attorneys.attorneys_rejected_patents_count` A
    WHERE
    NOT EXISTS
    (SELECT R.atty_registration_number FROM `ml-project2-333508.attorneys.attorneys_approved_patents_count` R
    WHERE R.atty_registration_number = A.atty_registration_number)

# attorneys_data
(SELECT * FROM `ml-project2-333508.attorneys.attorneys_data_mixed`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.attorneys.attorneys_data_approved`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.attorneys.attorneys_data_rejected`)


Now we have a table (`attorneys_data`) that contains the following columns: 
- `atty_registration_number`
- `total_patents` (patents approved + patents rejected for non-novelty at first attempt)
- `approval_rate` (percentage of approved patents)

We now build the features for each approved and rejected application

In [ ]:
# features_attorneys_approved
SELECT A.application_number, AVG(I.approval_rate) as attorney_avg_approval_rate, AVG(I.total_patents) as attorney_avg_total_patents, COUNT(*) as attorneys_number
    FROM `ml-project2-333508.office_actions.application_data_approved_patents_filtered` A, 
    `ml-project2-333508.inventors.attorneys_data` I,
    `ml-project2-333508.inventors.attorneys_approved_patents` I_A
    WHERE A.application_number = I_A.application_number AND I_A.atty_registration_number = I.atty_registration_number
    GROUP BY A.application_number
    ORDER BY A.application_number

# features_attorneys_rejected
SELECT A.application_number, AVG(I.approval_rate) as attorney_avg_approval_rate, AVG(I.total_patents) as attorney_avg_total_patents, COUNT(*) as attorneys_number
    FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty_with_claim` A, 
    `ml-project2-333508.inventors.attorneys_data` I,
    `ml-project2-333508.inventors.attorneys_rejected_patents` I_A
    WHERE A.application_number = I_A.application_number AND I_A.atty_registration_number = I.atty_registration_number
    GROUP BY A.application_number
    ORDER BY A.application_number

### Combine the features
We have 4 different tables: `features_inventors_approved`, `features_inventors_rejected`, `features_attorneys_approved`, `features_attorneys_rejected`. Let's combine them. 

In [ ]:
# features_approved
SELECT A.application_number, "1" as approved, A.examiner_art_unit, A.uspc_class, A.uspc_subclass, I.inventors_number, ROUND(I.inventor_avg_approval_rate, 2) as inventors_avg_approval_rate, ROUND(I.inventor_avg_total_patents, 2) as inventors_avg_toal_patents, T.attorney_number as attorneys_number, ROUND(T.attorney_avg_total_patents, 2) as attorneys_avg_total_patents, ROUND(T.attorney_avg_approval_rate, 2) as attorneys_avg_approval_rate
    FROM `ml-project2-333508.office_actions.application_data_approved_patents_filtered` A,
    `ml-project2-333508.features.features_inventors_approved` I,
    `ml-project2-333508.features.features_attorneys_approved` T
    WHERE A.application_number = I.application_number AND A.application_number = T.application_number
    ORDER BY A.application_number

# features_rejected
SELECT A.application_number, "0" as approved, A.examiner_art_unit, A.uspc_class, A.uspc_subclass, I.inventors_number, ROUND(I.inventor_avg_approval_rate, 2) as inventors_avg_approval_rate, ROUND(I.inventor_avg_total_patents, 2) as inventors_avg_toal_patents, T.attorney_number as attorneys_number, ROUND(T.attorney_avg_total_patents, 2) as attorneys_avg_total_patents, ROUND(T.attorney_avg_approval_rate, 2) as attorneys_avg_approval_rate
    FROM `ml-project2-333508.office_actions.application_data_rejected_non-novelty_with_claim` A,
    `ml-project2-333508.features.features_inventors_rejected` I,
    `ml-project2-333508.features.features_attorneys_rejected` T
    WHERE A.application_number = I.application_number AND A.application_number = T.application_number
    ORDER BY A.application_number

Some applications don't contain information about the inventors or attorneys. We now filter patent claims again in order to remove those applications. 

In [ ]:
# features_approved_claims (we substitute pat_no with application_number and filter again)
SELECT F.application_number, P.claim_no AS claim_number, P.claim_txt AS claim_text FROM `ml-project2-333508.features.features_approved` F, `ml-project2-333508.office_actions.application_data_approved_patents_filtered` A, `ml-project2-333508.patent_claims_fulltext.approved_patents_claims_filtered_cleaned` P
    WHERE F.application_number = A.application_number AND P.pat_no = A.patent_number
    ORDER BY F.application_number, CAST(P.claim_no AS INT)

# features_rejected_claims 
SELECT F.application_number, P.claim_no AS claim_number, P.claim_txt AS claim_text FROM `ml-project2-333508.features.features_rejected` F,  `ml-project2-333508.patent_claims_fulltext.rejected_patents_non-novelty_claims_cleaned` P
    WHERE P.appl_id = F.application_number
    ORDER BY F.application_number, CAST(P.claim_no AS INT)

We finally combine all the features

In [ ]:
# features 
SELECT * FROM
(SELECT * FROM `ml-project2-333508.features.features_approved`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.features.features_rejected`)
ORDER BY application_number


# features_claims
SELECT * FROM
(SELECT * FROM `ml-project2-333508.features.features_approved_claims`)
UNION ALL 
(SELECT * FROM `ml-project2-333508.features.features_rejected_claims`)
ORDER BY application_number